In [1]:
!pip install youtube-transcript-api

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 59.6 MB/s eta 0:00:00


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi

video_ids = ['3pp-RUaUhvY', 'LvrhaYhEb4c', 'hzcD1T_-XK4', 'fduEwKC0OKw', 'X-e2FOhL13Y', 'ZLcAtZIkbBw', '3In3LgSiCcM', 'YrUjuH8zqsY', 
              'wi9uSGrRC9o', 'e4zwTPFkrFw', 'J-p955Needw', 'ENRfckcQ-9Y', 'O-9wllzPEF8', 'axgcspwfXPw', 'Pxlq4KoYEkI', 'Onyn-Y3Ku7I',
              'IF0biMM_2VA', 'bNGYW4CSlZg', 'sa3HX5O1qEA', '_0v98iq-mRs', '-9H0A62JaZY', '1kZq5AdkCyY', 'RM28oR1c4xU', '3SYrf9NAJZE',
              'MYqTovzZW8U', '-GEAyCIgmI8', 'mKUXApaJuSE', 'o1x2iYtRzWI', '2Ika8e2Yr5w', 'Cj1Iin-UKzk', 'zYgjx1qFpkc', 'nFD7hgmw9xs',]

languages = ['ko', 'en']

transcript_list  = list(YouTubeTranscriptApi.get_transcripts(video_ids=video_ids, languages=languages))
transcript_list.pop()

with open('script_test.txt', 'w', encoding='utf-8') as f:
    for transcript_id in transcript_list:
        for id in video_ids:
            for transcript in transcript_id[id]:
                text = transcript['text']

                f.write(text)
                f.write('\n')

# []: 자막
# -: 대화

print('Complete!')

Complete!


### 퍼피 교육에 있는 모든 영상 크롤링 및 영상 제목을 포함한 csv파일 형태로

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import time

# --- 1. Selenium 드라이버 설정 ---
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 창 안 띄우고 싶으면 이 줄 주석 해제
driver = webdriver.Chrome(options=options)

PLAYLIST_URL = 'https://www.youtube.com/playlist?list=PLVh3TM0B0Wtl67fiuSL5xNEc0uZYvEB18'
driver.get(PLAYLIST_URL)

# 페이지 로딩 기다리기
time.sleep(3)

# --- 2. BeautifulSoup으로 영상 제목/ID 가져오기 ---

soup = BeautifulSoup(driver.page_source, "html.parser")

video_data = []

for a_tag in soup.select('a#video-title'):
    href = a_tag.get('href')
    if href and 'watch?v=' in href:
        video_id = href.split('watch?v=')[1].split('&')[0]
        title = a_tag.text.strip()
        video_data.append({
            'title': title,
            'video_id': video_id
        })

print(f"총 {len(video_data)}개의 영상 발견")

# 드라이버 종료
driver.quit()

# --- 3. YouTubeTranscriptApi로 자막 가져오기 ---

final_result = []

for idx, video in enumerate(video_data, 1):
    video_id = video['video_id']
    title = video['title']

    print(f"[{idx}] {title} ({video_id}) 자막 가져오는 중...")

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
        text_all = []

        for t in transcript:
            text = t['text']
            if text and not text.startswith('[') and not text.startswith('-'):
                text_all.append(text)

        script_text = ' '.join(text_all)

        final_result.append({
            'title': title,
            'video_id': video_id,
            'script': script_text
        })

    except Exception as e:
        print(f"[ERROR] {video_id}: {e}")
        continue

    time.sleep(0.5)

# --- 4. CSV 저장 ---
df = pd.DataFrame(final_result)
csv_filename = 'youtube_pupy.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ CSV 저장 완료: {csv_filename}")


총 36개의 영상 발견
[1] 얘를 어떻게 교육합니까? 발 닳습니다 [퍼피교육] (3pp-RUaUhvY) 자막 가져오는 중...
[2] 귀여우면 다야? 말 안 듣는 솜뭉치 훈련한 강형욱 [퍼피교육] (LvrhaYhEb4c) 자막 가져오는 중...
[3] 이 강아지는 이제 제 겁니다. 강형욱이 반한 예쁜 강아지 [퍼피교육] (hzcD1T_-XK4) 자막 가져오는 중...
[4] 강형욱 만나고 고장난 못생긴 노을이 [주니어교육] (fduEwKC0OKw) 자막 가져오는 중...
[5] 이름이 '짜장 곱백'이 뭐야ㅠ 넘모 귀엽잖아 [퍼피교육] (X-e2FOhL13Y) 자막 가져오는 중...
[6] 왕크니까 왕귀여운 초대형 애기 강아지 교육하기 [퍼피교육] (ZLcAtZIkbBw) 자막 가져오는 중...
[7] 얘를 무슨.. 배변 훈련을 해요. 실수하면 그냥 닦아주세요! [퍼피교육] (wi9uSGrRC9o) 자막 가져오는 중...
[8] "오늘 안 만났으면 1년 뒤에 개훌륭에서 봤을 거 같아요" 문제견 조기 발견한 강형욱 [퍼피교육] (3In3LgSiCcM) 자막 가져오는 중...
[9] “이미 앉았는데요.” 다리가 짧아서 슬픈 애기 닥스훈트[퍼피교육] (YrUjuH8zqsY) 자막 가져오는 중...
[10] 애기 리트리버가 물 먹는 법 [퍼피 교육] (e4zwTPFkrFw) 자막 가져오는 중...
[11] 비 오는 날 마중 나온 목화 강아지 [퍼피교육] (J-p955Needw) 자막 가져오는 중...
[12] 강아지 밥상머리 교육하는 법 [퍼피교육] (ENRfckcQ-9Y) 자막 가져오는 중...
[13] 천재 도베르만 등장 [퍼피교육] (O-9wllzPEF8) 자막 가져오는 중...
[14] 예민한 솜사탕 포메와 친해지기 [퍼피교육] (axgcspwfXPw) 자막 가져오는 중...
[15] 오랜만에 만난 세상 해맑은 강아지 [퍼피교육] (Pxlq4KoYEkI) 자막 가져오는 중...
[16] 귀여움이 두 배인 시바이누 두 마리 교육하기 [퍼피교육] (Onyn-Y3K

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import time

# --- 1. Selenium 드라이버 설정 ---
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# ✅ 수정된 URL: 재생목록 전용
PLAYLIST_URL = 'https://www.youtube.com/playlist?list=PLVh3TM0B0WtksY4ZQVNdD0aTG1Qm1mQNM'
driver.get(PLAYLIST_URL)
time.sleep(3)

# --- 2. 스크롤 내려서 모든 영상 로딩 ---
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# --- 3. BeautifulSoup으로 영상 제목/ID 가져오기 ---
soup = BeautifulSoup(driver.page_source, "html.parser")

video_data = []
for a_tag in soup.select('a.pl-video-title-link, a#video-title'):
    href = a_tag.get('href')
    if href and 'watch?v=' in href:
        video_id = href.split('watch?v=')[1].split('&')[0]
        title = a_tag.text.strip()
        if title:  # 빈 제목 필터링
            video_data.append({'title': title, 'video_id': video_id})

print(f"총 {len(video_data)}개의 영상 발견")

driver.quit()

# --- 4. 자막 가져오기 ---
final_result = []
for idx, video in enumerate(video_data, 1):
    video_id = video['video_id']
    title = video['title']
    print(f"[{idx}] {title} ({video_id}) 자막 가져오는 중...")

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
        text_all = [
            t['text'] for t in transcript
            if t['text'] and not t['text'].startswith('[') and not t['text'].startswith('-')
        ]
        script_text = ' '.join(text_all)
        final_result.append({'title': title, 'video_id': video_id, 'script': script_text})
    except Exception as e:
        print(f"[ERROR] {video_id}: {e}")
        continue

    time.sleep(0.5)

# --- 5. CSV 저장 ---
df = pd.DataFrame(final_result)
csv_filename = 'youtube_dog_dictionary.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료: {csv_filename}")


총 84개의 영상 발견
[1] 통나무 강아지 | 견종백과 노리치 테리어편 (hw_LqApxMt0) 자막 가져오는 중...
[2] 개훌륭 단골 출연견. 드디어 나왔습니다 | 견종백과 로트와일러편 (s4GuF4HMTlc) 자막 가져오는 중...
[3] 쭉쭉 늘어납니다~! (like 모짜렐라)  | 견종백과 샤페이편 (scowNj7Pyak) 자막 가져오는 중...
[4] 신이라 불리는 강아지. 이집트 벽화에도 존재함ㄷㄷ | 견종백과 살루키편 (BDHZh_Y8jck) 자막 가져오는 중...
[5] 내가 왕이 될 상인가. 성질머리에 비해 몸이 작아 슬픈 미니핀 | 견종백과 미니어처 핀셔편 (jbs_A0FT16c) 자막 가져오는 중...
[6] 유리몸 강아지. 내 지갑도 와장창... | 견종백과 잉글리쉬 불독편 (iQD00qkGya4) 자막 가져오는 중...
[7] 지랄총량 따위는 없는 무한체력 악마견 | 견종백과 잭 러셀 테리어편 (NHGkLb5ifu4) 자막 가져오는 중...
[8] 귀로 땅바닥 쓸고 다님 | 견종백과 바셋하운드편 (soIoUq72J3w) 자막 가져오는 중...
[9] 역대 최고 몸무게와 크기를 자랑하는 강형욱의 드림견 등장 | 견종백과 아이리시 울프 하운드편 (K4SCokZRZG8) 자막 가져오는 중...
[10] 귀털 자르면 안 되세요. 얼마나 소중한 건데. | 견종백과 카발리에 킹 찰스 스파니엘편 (-uX4Sa09TZ8) 자막 가져오는 중...
[11] 형이라 부를게요.... | 견종백과 아메리칸 불리편 (e-k1EnAWBS4) 자막 가져오는 중...
[12] 앞이 안 보이니깤! | 견종백과 삽살개편 (2tNFHa3LWT4) 자막 가져오는 중...
[13] (리트리버 아님)1박2일 마스코트였던 '그레이트 피레니즈' 두둥등장 | 견종백과 그레이트 피레니즈편 (Lhi7tJzRKXk) 자막 가져오는 중...
[14] 누구나 마트에서 한 번쯤 봤을 '그 강아지' | 견종백과 웨스트 하이랜드 화이트 테리어편 (gLcYWT4Z0gI) 자막 가져오는 중...

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# --- 1. Selenium 드라이버 설정 ---
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 창 안 띄우고 싶으면 이 줄 주석 해제
driver = webdriver.Chrome(options=options)

PLAYLIST_URL = 'https://www.youtube.com/playlist?list=PLVh3TM0B0Wtl67fiuSL5xNEc0uZYvEB18'
driver.get(PLAYLIST_URL)

# 페이지 로딩 기다리기
time.sleep(3)

# --- 2. BeautifulSoup으로 영상 제목/ID 가져오기 ---
soup = BeautifulSoup(driver.page_source, "html.parser")
video_data = []

for a_tag in soup.select('a#video-title'):
    href = a_tag.get('href')
    if href and 'watch?v=' in href:
        video_id = href.split('watch?v=')[1].split('&')[0]
        title = a_tag.text.strip()
        video_data.append({
            'title': title,
            'video_id': video_id,
            'URL': f'https://www.youtube.com/watch?v={video_id}',
            'image': f'https://img.youtube.com/vi/{video_id}/0.jpg'
        })

print(f"총 {len(video_data)}개의 영상 발견")

driver.quit()

# --- 3. CSV 저장 ---
df = pd.DataFrame(video_data)
csv_filename = 'youtube_video_links.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ CSV 저장 완료: {csv_filename}")


총 36개의 영상 발견

✅ CSV 저장 완료: youtube_video_links.csv


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# --- 1. Selenium 드라이버 설정 ---
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# ✅ 플레이리스트 URL
PLAYLIST_URL = 'https://www.youtube.com/playlist?list=PLVh3TM0B0WtksY4ZQVNdD0aTG1Qm1mQNM'
driver.get(PLAYLIST_URL)
time.sleep(3)

# --- 2. 스크롤 내려서 모든 영상 로딩 ---
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# --- 3. BeautifulSoup으로 영상 제목/ID 가져오기 ---
soup = BeautifulSoup(driver.page_source, "html.parser")

video_data = []
for a_tag in soup.select('a.pl-video-title-link, a#video-title'):
    href = a_tag.get('href')
    if href and 'watch?v=' in href:
        video_id = href.split('watch?v=')[1].split('&')[0]
        title = a_tag.text.strip()
        if title:
            video_data.append({
                'title': title,
                'video_id': video_id,
                'URL': f'https://www.youtube.com/watch?v={video_id}',
                'image': f'https://img.youtube.com/vi/{video_id}/0.jpg'
            })

print(f"총 {len(video_data)}개의 영상 발견")

# --- 4. CSV 저장 ---
df = pd.DataFrame(video_data)
csv_filename = 'youtube_dog_dictionary_simple.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ CSV 저장 완료: {csv_filename}")


총 85개의 영상 발견

✅ CSV 저장 완료: youtube_dog_dictionary_simple.csv
